In [4]:
import os
import pandas as pd
from tqdm import tqdm
from googleapiclient.discovery import build
from google.cloud import bigquery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/tanju/Desktop/upheld-momentum-463013-v7-910c47d3ace5.json"

PROJECT_ID = "upheld-momentum-463013-v7"
DATASET_ID = "dbt_tdereli"
TARGET_DATASET = "dbt_tdereli"
SOURCE_TABLE = "stg_youtube_trending"
DEST_TABLE = "channel_info_enriched" 


YOUTUBE_API_KEY = os.getenv("YOUTUBE_API_KEY")
youtube = build("youtube", "v3", developerKey=YOUTUBE_API_KEY)

client = bigquery.Client(project=PROJECT_ID)


query = f"""
    SELECT DISTINCT video_id
    FROM `{PROJECT_ID}.{DATASET_ID}.{SOURCE_TABLE}`
"""
video_ids = [row["video_id"] for row in client.query(query)]
print(f"Found {len(video_ids)} unique video IDs.")


def fetch_channel_info(batch_ids):
    try:
        response = youtube.videos().list(
            part="snippet",
            id=",".join(batch_ids)
        ).execute()

        returned_items = response.get("items", [])
        returned_ids = {item["id"] for item in returned_items}
        missing_ids = set(batch_ids) - returned_ids

        if missing_ids:
            print(f"[WARN] Missing video IDs from API: {missing_ids}")

        # Build a dict for quick lookup of returned videos
        item_lookup = {item["id"]: item for item in returned_items}
        
        result = []
        for vid in batch_ids:
            if vid in item_lookup:
                item = item_lookup[vid]
                result.append({
                    "video_id": vid,
                    "channel_id": item["snippet"]["channelId"],
                    "channel_title": item["snippet"]["channelTitle"],
                })
            else:
                # Still add the row with Nones so it's aligned
                result.append({
                    "video_id": vid,
                    "channel_id": None,
                    "channel_title": None,
                })

        return result

    except Exception as e:
        print(f"[ERROR] Failed fetching batch: {e}")
        return []




channel_info = []
BATCH_SIZE = 50
for i in tqdm(range(0, len(video_ids), BATCH_SIZE), desc="Fetching channel info"):
    batch = video_ids[i:i + BATCH_SIZE]
    channel_info.extend(fetch_channel_info(batch))

print(f"Successfully enriched {len(channel_info)} videos.")


df = pd.DataFrame(channel_info)




job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
    schema=[
        bigquery.SchemaField("video_id", "STRING"),
        bigquery.SchemaField("channel_id", "STRING"),
        bigquery.SchemaField("channel_title", "STRING"),
    ]
)

table_ref = f"{PROJECT_ID}.{TARGET_DATASET}.{DEST_TABLE}"
load_job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
load_job.result()

print(f"Uploaded enriched data to BigQuery table name `{DEST_TABLE}`.")

Found 8831 unique video IDs.


Fetching channel info:   4%|▍         | 7/177 [00:03<00:43,  3.87it/s]

[WARN] Missing video IDs from API: {'1XBMMwEpCeg'}


Fetching channel info:   6%|▌         | 11/177 [00:03<00:31,  5.35it/s]

[WARN] Missing video IDs from API: {'UmZYLCoPahE'}
[WARN] Missing video IDs from API: {'D7QxvWShyuY'}


...

[WARN] Missing video IDs from API: {'WPQIexo41lo'}
[WARN] Missing video IDs from API: {'qJHpux2dolU'}


Fetching channel info:  99%|█████████▉| 175/177 [00:36<00:00,  4.92it/s]

[WARN] Missing video IDs from API: {'Ww7Y_uNTFPQ'}


Fetching channel info: 100%|██████████| 177/177 [00:37<00:00,  4.77it/s]


Successfully enriched 8831 videos.
Uploaded enriched data to BigQuery table name `channel_info_enriched`.
